In [ ]:
# run against production 2024-12-19
from acdh_tei_pyutils.tei import TeiReader
from acdh_tei_pyutils.utils import get_xmlid
from acdh_xml_pyutils.xml import NSMAP
from normdata.utils import import_from_normdata
from tqdm import tqdm

In [ ]:
doc = TeiReader("https://emt.acdh-dev.oeaw.ac.at/listperson.xml")
domain = "kaiserin-eleonora"
col, _ = Collection.objects.get_or_create(name=domain)
col

In [ ]:
broken_gnd = []
for x in tqdm(doc.any_xpath(".//tei:person[@xml:id and ./tei:idno[@type='GND']]")):
    gnd_uri = x.xpath("./tei:idno[@type='GND']/text()", namespaces=NSMAP)[0]
    xmlid = get_xmlid(x)
    uri = "https://kaiserin-eleonora.oeaw.ac.at/{}.html".format(xmlid)
    entity = import_from_normdata(gnd_uri, "person")
    if entity:
        pmb_uri, _ = Uri.objects.get_or_create(uri=uri, domain=domain)
        pmb_uri.entity = entity
        pmb_uri.save()
        entity.collection.add(col)
    else:
        broken_gnd.append(gnd_uri)

In [ ]:
print(broken_gnd)